<a href="https://colab.research.google.com/github/Joelss23/Actividades/blob/main/Caso_Practica_Analitica_Espacial_Joel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Cargar Librerias

In [ ]:
#Librerías necesarias
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import folium
from folium import LayerControl
from google.colab import files

#2. Carga de Datos

In [ ]:
#Cargar Archivos
uploaded = files.upload()

In [ ]:
#Cargar oficinas turísticas
with open("oficinas_turisticas.json", encoding='utf-8') as f:
    oficinas_json = json.load(f)

In [ ]:
#Extraccion de datos
oficinas_data = oficinas_json['@graph']

#Normalizacion para latitud y longitud
oficinas_df = pd.json_normalize(
    oficinas_data,
    sep="_"
)

In [ ]:
#Convertir GeoDataFrame
oficinas_gdf = gpd.GeoDataFrame(
    oficinas_df,
    geometry=gpd.points_from_xy(
        oficinas_df['location_longitude'],
        oficinas_df['location_latitude']
    ),
    crs="EPSG:4326"
)

In [ ]:
#Cargar estaciones de metro
metro_gdf = gpd.read_file("M4_Estaciones.geojson")

metro_gdf = metro_gdf.to_crs(epsg=4326)

#3. Visualizacion

In [ ]:
#Crear mapa base centrado en Madrid
mapa = folium.Map(location=[40.4168, -3.7038], zoom_start=13)

In [ ]:
#Añadir oficinas turísticas
folium.GeoJson(
    oficinas_gdf,
    name="Oficinas Turísticas",
    tooltip=folium.GeoJsonTooltip(fields=["title", "address_street-address", "address_postal-code"]),
    marker=folium.CircleMarker(radius=6, color="red", fill=True, fill_opacity=0.8),
).add_to(mapa)

In [ ]:
#Añadir estaciones de metro
folium.GeoJson(
    metro_gdf,
    name="Estaciones de Metro",
    tooltip=folium.GeoJsonTooltip(fields=["DENOMINACION"]),  # <-- Corrección aquí
    marker=folium.CircleMarker(radius=4, color="blue", fill=True, fill_opacity=0.7),
).add_to(mapa)

In [ ]:
#Añadir control de capas
LayerControl().add_to(mapa)

In [ ]:
#Mostrar mapa
mapa